In [1]:
import csv
import json
import numpy as np
import pandas as pd
import re
import urllib.request
from bs4 import BeautifulSoup
from datetime import date

In [2]:
class FinancialsDownloader(object):
    u"""Downloads financials from http://financials.morningstar.com/
    """

    def __init__(self, table_prefix = u'morningstar_'):
        u"""Constructs the FinancialsDownloader instance.

        :param table_prefix: Prefix of the MySQL tables.
        """
        self._table_prefix = table_prefix

    def download(self, ticker, conn = None):
        u"""Downloads and returns a dictionary containing pandas.DataFrames
        representing the financials (i.e. income statement, balance sheet,
        cash flow) for the given Morningstar ticker. If the MySQL connection
        is specified then the downloaded financials are uploaded to the MySQL
        database.

        :param ticker: Morningstar ticker.
        :param conn: MySQL connection.
        :return Dictionary containing pandas.DataFrames representing the
        financials for the given Morningstar ticker.
        """
        result = {}

        ##########################
        # Error Handling
        ##########################

        # Empty String
        if len(ticker) == 0:
            raise ValueError("You did not enter a ticker symbol.  Please"
                             " try again.")

        for report_type, table_name in [
                (u'is', u'income_statement'),
                (u'bs', u'balance_sheet'),
                (u'cf', u'cash_flow')]:
            frame = self._download(ticker, report_type)
            result[table_name] = frame
            if conn:
                self._upload_frame(
                    frame, ticker, self._table_prefix + table_name, conn)
        if conn:
            self._upload_unit(ticker, self._table_prefix + u'unit', conn)
        result[u'period_range'] = self._period_range
        result[u'fiscal_year_end'] = self._fiscal_year_end
        result[u'currency'] = self._currency
        return result

    def _download(self, ticker, report_type):
        u"""Downloads and returns a pandas.DataFrame corresponding to the
        given Morningstar ticker and the given type of the report.

        :param ticker: Morningstar ticker.
        :param report_type: Type of the report ('is', 'bs', 'cf').
        :return  pandas.DataFrame corresponding to the given Morningstar ticker
        and the given type of the report.
        """
        url = (r'http://financials.morningstar.com/ajax/' +
               r'ReportProcess4HtmlAjax.html?&t=' + ticker +
               r'&region=usa&culture=en-US&cur=USD' +
               r'&reportType=' + report_type + r'&period=12' +
               r'&dataType=A&order=asc&columnYear=5&rounding=3&view=raw')
        with urllib.request.urlopen(url) as response:
            json_text = response.read().decode(u'utf-8')

            ##############################
            # Error Handling
            ##############################

            # Wrong ticker
            if len(json_text)==0:
                raise ValueError("MorningStar cannot find the ticker symbol "
                                 "you entered or it is INVALID. Please try "
                                 "again.")

            json_data = json.loads(json_text)
            result_soup = BeautifulSoup(json_data[u'result'],u'html.parser')
            return self._parse(result_soup)

    def _parse(self, soup):
        u"""Extracts and returns a pandas.DataFrame corresponding to the
        given parsed HTML response from financials.morningstar.com.

        :param soup: Parsed HTML response by BeautifulSoup.
        :return pandas.DataFrame corresponding to the given parsed HTML response
        from financials.morningstar.com.
        """
        # Left node contains the labels.
        left = soup.find(u'div', u'left').div
        # Main node contains the (raw) data.
        main = soup.find(u'div', u'main').find(u'div', u'rf_table')
        year = main.find(u'div', {u'id': u'Year'})
        self._year_ids = [node.attrs[u'id'] for node in year]
        period_month = pd.datetime.strptime(year.div.text, u'%Y-%m').month
        self._period_range = pd.period_range(
            year.div.text, periods=len(self._year_ids),
            # freq=pd.datetools.YearEnd(month=period_month))
            freq = pd.tseries.offsets.YearEnd(month=period_month))
        unit = left.find(u'div', {u'id': u'unitsAndFiscalYear'})
        self._fiscal_year_end = int(unit.attrs[u'fyenumber'])
        self._currency = unit.attrs[u'currency']
        self._data = []
        self._label_index = 0
        self._read_labels(left)
        self._data_index = 0
        self._read_data(main)
        return pd.DataFrame(self._data,
                            columns=[u'parent_index', u'title'] + list(
                                self._period_range))

    def _read_labels(self, root_node, parent_label_index = None):
        u"""Recursively reads labels from the parsed HTML response.
        """
        for node in root_node:
            if node.has_attr(u'class') and u'r_content' in node.attrs[u'class']:
                self._read_labels(node, self._label_index - 1)
            if (node.has_attr(u'id') and
                    node.attrs[u'id'].startswith(u'label') and
                    not node.attrs[u'id'].endswith(u'padding') and
                    (not node.has_attr(u'style') or
                        u'display:none' not in node.attrs[u'style'])):
                label_id = node.attrs[u'id'][6:]
                label_title = (node.div.attrs[u'title']
                               if node.div.has_attr(u'title')
                               else node.div.text)
                self._data.append({
                    u'id': label_id,
                    u'index': self._label_index,
                    u'parent_index': (parent_label_index
                                     if parent_label_index is not None
                                     else self._label_index),
                    u'title': label_title})
                self._label_index += 1

    def _read_data(self, root_node):
        u"""Recursively reads data from the parsed HTML response.
        """
        for node in root_node:
            if node.has_attr(u'class') and u'r_content' in node.attrs[u'class']:
                self._read_data(node)
            if (node.has_attr(u'id') and
                    node.attrs[u'id'].startswith(u'data') and
                    not node.attrs[u'id'].endswith(u'padding') and
                    (not node.has_attr(u'style') or
                        u'display:none' not in node.attrs[u'style'])):
                data_id = node.attrs[u'id'][5:]
                while (self._data_index < len(self._data) and
                       self._data[self._data_index][u'id'] != data_id):
                    # In some cases we do not have data for all labels.
                    self._data_index += 1
                assert(self._data_index < len(self._data) and
                       self._data[self._data_index][u'id'] == data_id)
                for (i, child) in enumerate(node.children):
                    try:
                        value = float(child.attrs[u'rawvalue'])
                    except ValueError:
                        value = None
                    self._data[self._data_index][
                        self._period_range[i]] = value
                self._data_index += 1

    def _upload_frame(self, frame, ticker, table_name,
                      conn):
        u"""Uploads the given pandas.DataFrame to the MySQL database.

        :param frame: pandas.DataFrames to be uploaded.
        :param ticker: Morningstar ticker.
        :param table_name: Name of the MySQL table.
        :param conn: MySQL connection.
        """
        if not _db_table_exists(table_name, conn):
            _db_execute(self._get_db_create_table(table_name), conn)
        _db_execute(self._get_db_replace_values(
            ticker, frame, table_name), conn)

    def _upload_unit(self, ticker, table_name,
                     conn):
        u"""Uploads the fiscal_year_end and the currency to the MySQL database.

        :param ticker: Morningstar ticker.
        :param table_name: Name of the MySQL table.
        :param conn: MySQL connection.
        """
        if not _db_table_exists(table_name, conn):
            _db_execute(
                u'CREATE TABLE `%s` (\n' % table_name +
                u'  `ticker` varchar(50) NOT NULL\n' +
                u'    COMMENT "Exchange:Ticker",\n' +
                u'  `fiscal_year_end` int(10) unsigned NOT NULL\n' +
                u'    COMMENT  "Fiscal Year End Month",\n' +
                u'  `currency` varchar(50) NOT NULL\n' +
                u'    COMMENT "Currency",\n' +
                u'  PRIMARY KEY USING BTREE (`ticker`))\n' +
                u'ENGINE=MyISAM DEFAULT CHARSET=utf8', conn)
        _db_execute(
            u'REPLACE INTO `%s`\n' % table_name +
            u'  (`ticker`, `fiscal_year_end`, `currency`)\nVALUES\n' +
            u'("%s", %d, "%s")' % (
                ticker, self._fiscal_year_end, self._currency), conn)

    @staticmethod
    def _get_db_create_table(table_name):
        u"""Returns the MySQL CREATE TABLE statement for the given table_name.

        :param table_name: Name of the MySQL table.
        :return MySQL CREATE TABLE statement.
        """
        year = date.today().year
        year_range = xrange(year - 6, year + 2)
        columns = u',\n'.join(
            [u'  `year_%d` DECIMAL(20,5) DEFAULT NULL ' % year +
             u'COMMENT "Year %d"' % year
             for year in year_range])
        return (
            u'CREATE TABLE `%s` (\n' % table_name +
            u'  `ticker` VARCHAR(50) NOT NULL COMMENT "Exchange:Ticker",\n' +
            u'  `id` int(10) unsigned NOT NULL COMMENT "Id",\n' +
            u'  `parent_id` int(10) unsigned NOT NULL COMMENT "Parent Id",\n' +
            u'  `item` varchar(500) NOT NULL COMMENT "Item",\n' +
            u'%s,\n' % columns +
            u'  PRIMARY KEY USING BTREE (`ticker`, `id`),\n' +
            u'  KEY `ix_ticker` USING BTREE (`ticker`))\n' +
            u'ENGINE=MyISAM DEFAULT CHARSET=utf8')

    @staticmethod
    def _get_db_replace_values(ticker, frame,
                               table_name):
        u"""Returns the MySQL REPLACE INTO statement for the given
        Morningstar ticker and the corresponding pandas.DataFrame.

        :param ticker: Morningstar ticker.
        :param frame: pandas.DataFrame.
        :param table_name: Name of the MySQL table.
        :return MySQL REPLACE INTO statement.
        """
        columns = [u'`ticker`', u'`id`, `parent_id`, `item`'] + \
                  [u'`year_%d`' % period.year for period in
                   frame.columns[2:]]
        return (
            u'REPLACE INTO `%s`\n' % table_name +
            u'  (%s)\nVALUES\n' % u', '.join(columns) +
            u',\n'.join([u'("' + ticker + u'", %d, %d, "%s", ' %
                        (index, frame.ix[index, u'parent_index'],
                         frame.ix[index, u'title']) +
                        u', '.join(
                            [u'NULL' if np.isnan(frame.ix[index, period])
                             else u'%.5f' % frame.ix[index, period]
                             for period in frame.columns[2:]]) + u')'
                        for index in frame.index]))

In [3]:
def _db_table_exists(table_name, conn):
    u"""Helper method for checking whether the given MySQL table exists.

    :param table_name: Name of the MySQL table to be checked.
    :param conn: MySQL connection.
    :return True iff the given MySQL table exists.
    """
    cursor = conn.cursor()
    cursor.execute(u"""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'""".format(table_name))
    table_exists = cursor.fetchone()[0] == 1
    cursor.close()
    return table_exists


def _db_execute(query, conn):
    u"""Helper method for executing the given MySQL non-query.

    :param query: MySQL query to be executed.
    :param conn: MySQL connection.
    """
    cursor = conn.cursor()
    cursor.execute(query)
    cursor.close()

In [4]:
fd=FinancialsDownloader()

In [5]:
fd.download('XSES:CH8')

{'balance_sheet':     parent_index                                       title          2012  \
 0              0                                      Assets  1.126049e+09   
 1              0                              Current assets  7.006020e+08   
 2              1                                        Cash  1.153030e+08   
 3              2                   Cash and cash equivalents  1.049560e+08   
 4              2                      Short-term investments  1.034700e+07   
 5              2                                  Total cash  1.153030e+08   
 6              1                                 Receivables  2.546930e+08   
 7              1                                 Inventories  1.423640e+08   
 8              1                            Prepaid expenses  2.040000e+05   
 9              1                        Other current assets  1.880380e+08   
 10             1                        Total current assets  7.006020e+08   
 11             0                  

In [ ]:
fd._parse

Test downloads function:

In [8]:
import csv
import json
import numpy as np
import pandas as pd
import re
import urllib.request
from bs4 import BeautifulSoup
from datetime import date

In [13]:
def test_download(ticker):
    report_type = 'is'
    url = (r'http://financials.morningstar.com/ajax/' +
               r'ReportProcess4HtmlAjax.html?&t=' + ticker +
               r'&region=usa&culture=en-US&cur=USD' +
               r'&reportType=' + report_type + r'&period=12' +
               r'&dataType=A&order=asc&columnYear=5&rounding=3&view=raw')
    with urllib.request.urlopen(url) as response:
        json_text = response.read().decode(u'utf-8')

        ##############################
        # Error Handling
        ##############################

        # Wrong ticker
        if len(json_text)==0:
            raise ValueError("MorningStar cannot find the ticker symbol "
                                 "you entered or it is INVALID. Please try "
                                 "again.")

        json_data = json.loads(json_text)
        result_soup = BeautifulSoup(json_data[u'result'],u'html.parser')
        return parse(result_soup)
    
def parse(soup):
    u"""Extracts and returns a pandas.DataFrame corresponding to the
    given parsed HTML response from financials.morningstar.com.

    param soup: Parsed HTML response by BeautifulSoup.
    :return pandas.DataFrame corresponding to the given parsed HTML response
    from financials.morningstar.com.
    """
    # Left node contains the labels.
    left = soup.find(u'div', u'left').div
    # Main node contains the (raw) data.
    main = soup.find(u'div', u'main').find(u'div', u'rf_table')
    year = main.find(u'div', {u'id': u'Year'})
    year_ids = [node.attrs[u'id'] for node in year]
    period_month = pd.datetime.strptime(year.div.text, u'%Y-%m').month
    period_range = pd.period_range(
            year.div.text, periods=len(year_ids),
        # freq=pd.datetools.YearEnd(month=period_month))
        freq = pd.tseries.offsets.YearEnd(month=period_month))
    unit = left.find(u'div', {u'id': u'unitsAndFiscalYear'})
    fiscal_year_end = int(unit.attrs[u'fyenumber'])
    currency = unit.attrs[u'currency']
    data = []
    label_index = 0
    read_labels(left)
    data_index = 0
    read_data(main)
    return pd.DataFrame(data,
                            columns=[u'parent_index', u'title'] + list(
                                self._period_range))

In [14]:
test1 = test_download('XSES:CH8',)

NameError: name 'read_labels' is not defined